In [1]:
!git clone https://github.com/Thiggel/Monotonic-Early-Exit.git

fatal: destination path 'Monotonic-Early-Exit' already exists and is not an empty directory.


In [2]:
%cd Monotonic-Early-Exit/

/content/Monotonic-Early-Exit


In [3]:
!git checkout getting_classifiers_to_run

Already on 'getting_classifiers_to_run'
Your branch is up to date with 'origin/getting_classifiers_to_run'.


In [1]:
!pip install -r requirements.txt

  Using cached datasets-2.19.1-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-macosx_10_9_x86_64.whl.metadata (7.7 kB)
  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached py7zr-0.21.0-py3-none-any.whl.metadata (17 kB)
  Using cached sacrebleu-2.4.2-py3-none-any.whl.metadata (58 kB)
  Using cached transformers-4.28.1-py3-none-any.whl.metadata (109 kB)
  Using cached protobuf-3.20.1-py2.py3-none-any.whl.metadata (720 bytes)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp312-cp312-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata 

Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.5 MB/s eta 0:00:000:00:01
Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
Using cached sentencepiece-0.2.0-cp312-cp312-macosx_10_9_x86_64.whl (1.2 MB)
Using cached protobuf-3.20.1-py2.py3-none-any.whl (162 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:004.0 MB/s eta 0:00:01
Using cached py7zr-0.21.0-py3-none-any.whl (67 kB)
Using cached sacrebleu-2.4.2-py3-none-any.whl (106 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.4 MB/s eta 0:00:000:00:010:00:01
Using cached Brotli-1.1.0-cp312-cp312-macosx_10_9_x86_64.whl (446 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached inflate64-1.0.0-cp312-cp312-macosx_10_9_x86_64.whl (36 kB)
Using cached multivolumefile-0.2.3-py3-none-any.whl (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 9.7 MB/s eta 0:00:000m eta 0:00:010:00:01
Using cached p

Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
CUDA_VISIBLE_DEVICES=0

In [36]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_summarization.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config_name "3.0.0" \
    --output_dir ./save/cnndm_t5_large/ \
    --deploy_scenario True \
    --use_synchronize True \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 1 \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "summarize: " \
    --num_train_epochs 10 \
    --output_hidden_states_decoder True \
    --intermediate_loss_fn weighted_ce \
    --use_early_exit True \
    --exit_conf_type last_three_hiddens_classifier \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5


2024-05-21 01:49:02.294219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 01:49:02.294273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 01:49:02.296313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 01:49:04.103085: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 01:49:07 - WARNING - __main__ -

In [37]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_summarization.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config_name "3.0.0" \
    --output_dir ./save/cnndm_t5_large/ \
    --deploy_scenario True \
    --use_synchronize True \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 1 \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "summarize: " \
    --num_train_epochs 10 \
    --output_hidden_states_decoder True \
    --intermediate_loss_fn weighted_ce \
    --use_early_exit True \
    --exit_conf_type softmax \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 01:50:10.410698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 01:50:10.410749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 01:50:10.417062: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 01:50:12.402105: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 01:50:16 - WARNING - __main__ -

In [38]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_summarization.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config_name "3.0.0" \
    --output_dir ./save/cnndm_t5_large/ \
    --deploy_scenario True \
    --use_synchronize True \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 1 \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "summarize: " \
    --num_train_epochs 10 \
    --output_hidden_states_decoder True \
    --intermediate_loss_fn weighted_ce \
    --use_early_exit True \
    --exit_conf_type hidden_state_saturation \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 01:53:42.174213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 01:53:42.174267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 01:53:42.175697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 01:53:43.936173: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 01:53:47 - WARNING - __main__ -

In [39]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_summarization.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config_name "3.0.0" \
    --output_dir ./save/cnndm_t5_large/ \
    --deploy_scenario True \
    --use_synchronize True \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 1 \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "summarize: " \
    --num_train_epochs 10 \
    --output_hidden_states_decoder True \
    --intermediate_loss_fn weighted_ce \
    --use_early_exit True \
    --exit_conf_type last_three_top_prob_heuristic \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 01:57:15.961545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 01:57:15.961596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 01:57:15.963523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 01:57:17.177638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 01:57:20 - WARNING - __main__ -

In [42]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit False \
    --source_lang "de" \
    --target_lang "en"

2024-05-21 03:15:12.586579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:15:12.586631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:15:12.588065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:15:13.819236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:15:18 - WARNING - __main__ -

In [43]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit True \
    --source_lang "de" \
    --target_lang "en" \
    --intermediate_loss_fn weighted_ce \
    --exit_conf_type meta \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 03:19:30.834268: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:19:30.834321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:19:30.835923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:19:32.056077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:19:36 - WARNING - __main__ -

In [44]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit True \
    --source_lang "de" \
    --target_lang "en" \
    --intermediate_loss_fn weighted_ce \
    --exit_conf_type recurrent_classifier \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 03:22:56.915156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:22:56.915216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:22:56.917120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:22:58.802893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:23:03 - WARNING - __main__ -

In [45]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit True \
    --source_lang "de" \
    --target_lang "en" \
    --intermediate_loss_fn weighted_ce \
    --exit_conf_type last_three_hiddens_classifier \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 03:24:01.008129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:24:01.008180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:24:01.009965: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:24:03.297848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:24:08 - WARNING - __main__ -

In [46]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit True \
    --source_lang "de" \
    --target_lang "en" \
    --intermediate_loss_fn weighted_ce \
    --exit_conf_type softmax \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 03:25:06.826232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:25:06.826285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:25:06.828142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:25:08.671527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:25:14 - WARNING - __main__ -

In [47]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit True \
    --source_lang "de" \
    --target_lang "en" \
    --intermediate_loss_fn weighted_ce \
    --exit_conf_type hidden_state_saturation \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 03:28:36.685752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:28:36.685810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:28:36.692269: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:28:38.714724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:28:44 - WARNING - __main__ -

In [48]:
!python -m torch.distributed.run --nproc_per_node=1 \
    src/run_translation.py \
    --model_name_or_path t5-large \
    --do_eval \
    --dataset_name iwslt2017 \
    --dataset_config_name iwslt2017-de-en \
    --output_dir ./save/iwslt_t5_large/ \
    --per_device_eval_batch_size 1 \
    --deploy_scenario True \
    --use_synchronize True \
    --overwrite_output_dir \
    --predict_with_generate \
    --source_prefix "translate German to English: " \
    --output_hidden_states_decoder True \
    --use_early_exit True \
    --source_lang "de" \
    --target_lang "en" \
    --intermediate_loss_fn weighted_ce \
    --exit_conf_type last_three_top_prob_heuristic \
    --exit_position_temp 4 \
    --exit_conf_threshold 0.5

2024-05-21 03:31:02.663206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 03:31:02.663262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 03:31:02.664666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 03:31:04.014086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
05/21/2024 03:31:08 - WARNING - __main__ -